In [1]:
import os
import io
import time
import json
import urllib
import math 
import selenium
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
pd.set_option('max_colwidth', 400)

In [2]:
start_directory = os.getcwd()
start_directory

'C:\\Users\\riley\\Documents\\RL Overlays\\TritonRL-ScoreBot'

In [3]:
def sleepy(sec):
    time.sleep(sec)
    return

In [4]:
# Directory Setup

# Set up the playercard folder, if it doesnt exist, create one. 
filepath = start_directory + '\\' + 'playercards' + '\\'
formatted_fp = filepath.replace('\\', '/')
if not os.path.exists(formatted_fp): 
    print('Creating local playercards directory')
    os.mkdir(formatted_fp) 

Creating local playercards directory


In [6]:
# Selenium Webdriver Initialization
chrome_options = webdriver.ChromeOptions()
prefs = {"download.default_directory":filepath}
chrome_options.add_experimental_option('prefs', prefs)
browser = webdriver.Chrome(chrome_options=chrome_options)


c:\users\riley\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [6]:
# Get to FIFA card generator webpage
browser.get('https://www.fifarosters.com/create-card')

In [7]:
# ALL XPATHS NEEDED #
official_cards = '//*[@id="card_selector_btn"]'
# 5 categories of playercards to cateogrize player distribution
tiered_playercards = {
    'bronze':'//*[@id="card_selector_modal_21"]/div/div/div[2]/div/div[4]/a',
    'silver':'//*[@id="card_selector_modal_21"]/div/div/div[2]/div/div[5]/a',
    'gold':'//*[@id="card_selector_modal_21"]/div/div/div[2]/div/div[6]/a',
    'champ':'//*[@id="card_selector_modal_21"]/div/div/div[2]/div/div[35]/a', 
    'legend':'//*[@id="card_selector_modal_21"]/div/div/div[2]/div/div[24]/a'
}

# Upload player desired image
upload_img = '//*[@id="form-image-type-upload"]'
choose_file = '//*[@id="form-card-upload-image"]'
# Player Information
input_name = '//*[@id="form-card-name"]'
overall_rating = '//*[@id="form-card-rating"]'
position = '//*[@id="form-card-position"]'
club_field = '//*[@id="form-card-club-text"]'
nation_rank = '//*[@id="form-card-nation-text"]'
#Attributes
offense = '//*[@id="form-card-attr1-text"]'
ostat = '//*[@id="form-card-attr1"]'
aggression = '//*[@id="form-card-attr2-text"]'
astat = '//*[@id="form-card-attr2"]'
winrate = '//*[@id="form-card-attr3-text"]'
wstat = '//*[@id="form-card-attr3"]'
defense = '//*[@id="form-card-attr4-text"]'
dstat = '//*[@id="form-card-attr4"]'
speed = '//*[@id="form-card-attr5-text"]'
sstat = '//*[@id="form-card-attr5"]'
ranking = '//*[@id="form-card-attr6-text"]'
rstat = '//*[@id="form-card-attr6"]'

# Formatting and download
thin_letters = '//*[@id="createCardForm"]/div[18]/div/div/div/label[2]'
download = '//*[@id="main_row"]/div[2]/div/div/button'

In [8]:
# Pictures of the ranks found in rocket league
rank_images = {
    'Supersonic Legend': 'https://cdn.discordapp.com/attachments/805954757704417360/805957358152187924/SSL_smol.png',
    'Grand Champion III': 'https://cdn.discordapp.com/attachments/805954757704417360/805957462124658688/GC3.png',
    'Grand Champion II': 'https://cdn.discordapp.com/attachments/805954757704417360/805957458328551505/GC2.png',
    'Grand Champion I': 'https://cdn.discordapp.com/attachments/805954757704417360/805957454037909534/GC1.png',
    'Champion III': 'https://cdn.discordapp.com/attachments/805954757704417360/805959195794997268/C3.png',
    'Champion II': 'https://cdn.discordapp.com/attachments/805954757704417360/805958920018591754/C2.png',
    'Champion I': 'https://cdn.discordapp.com/attachments/805954757704417360/805958913454768149/C1.png',
    'Diamond III': 'https://cdn.discordapp.com/attachments/805954757704417360/805959243307417622/D2.png',
    'Diamond II': 'https://cdn.discordapp.com/attachments/805954757704417360/805959240132067328/D1.png',
    'Diamond I': 'https://cdn.discordapp.com/attachments/805954757704417360/805959240132067328/D1.png',
    'Platinum III': 'https://cdn.discordapp.com/attachments/805954757704417360/805959311276376124/P3.png',
    'Platinum II': 'https://cdn.discordapp.com/attachments/805954757704417360/805959307153375242/P1.png',
    'Platinum I': 'https://cdn.discordapp.com/attachments/805954757704417360/805959307153375242/P1.png',
    'Gold III': 'https://cdn.discordapp.com/attachments/805954757704417360/805959370226794506/G3.png',
    'Gold II': 'https://cdn.discordapp.com/attachments/805954757704417360/805959370226794506/G2.png',
    'Gold I': 'https://cdn.discordapp.com/attachments/805954757704417360/805959368348532786/G1.png',
    'Silver III': 'https://cdn.discordapp.com/attachments/805954757704417360/805959373742145556/S3.png',
    'Silver II': 'https://cdn.discordapp.com/attachments/805954757704417360/805959373742145556/S2.png',
    'Silver I': 'https://cdn.discordapp.com/attachments/805954757704417360/805959373742145556/S1.png',
    'Bronze III': 'https://cdn.discordapp.com/attachments/805954757704417360/805959365206212638/B3.png',
    'Bronze II': 'https://cdn.discordapp.com/attachments/805954757704417360/805959362941550613/B2.png',
    'Bronze I': 'https://cdn.discordapp.com/attachments/805954757704417360/805959360575963166/B1.png'
}

# Default Images 
default_rank = 'https://cdn.discordapp.com/attachments/766119517503619097/805555338807345162/white-question-mark-emoji-removebg-preview.png' # Default Unknown Rank
default_logo = 'https://cdn.discordapp.com/attachments/764281194631397416/772652236249104384/tritonrl_logo-removebg-preview_1.png' # Triton Rocket League Logo
default_img = 'c:\\Users\\dmarc\\OneDrive\\Documents\\Github\\Personal Projects\\TritonRL-ScoreBot\\default_imgs\\default_player_avatar.png' # Locked Character Avatar


In [7]:
# Season Statistics
dataset = 'output/playercard_stats.csv'
df = pd.read_csv(dataset, index_col=0)
df = df.fillna('N/A')
df

,ranking,win rate,Overall,Offense,Defense,Aggression,Speed,discord_username,playercard_name,img_filepath,rank
name_index,,,,,,,,,,,
ix mini,0,0.53,99.00,100.00,58.41,75.58,95.73,Mini#6984,IX Mini,c:/Users/dmarc/OneDrive/Documents/Github/Personal Projects/TritonRL-ScoreBot/playercard_imgs/Mini#6984.png,N/A
smiggy,1,0.58,95.97,96.81,93.02,60.72,61.00,Smiggy#1206,smiggy,c:/Users/dmarc/OneDrive/Documents/Github/Personal Projects/TritonRL-ScoreBot/playercard_imgs/Smiggy#1206.png,Grand Champion I
shaunch,2,0.61,94.38,81.73,53.93,79.75,86.59,Shaunch#6025,Shaunch,c:/Users/dmarc/OneDrive/Documents/Github/Personal Projects/TritonRL-ScoreBot/playercard_imgs/Shaunch#6025.png,Champion I
invincible,3,0.39,93.06,81.61,35.49,100.00,76.94,N/A,N/A,N/A,N/A
alex,4,0.52,92.74,96.74,50.29,73.14,71.97,ohAlex_#9493,Alex,N/A,N/A
luisito,5,0.58,90.12,90.12,71.77,71.03,43.47,Luisito#0412,Luisito,c:/Users/dmarc/OneDrive/Documents/Github/Personal Projects/TritonRL-ScoreBot/playercard_imgs/Luisito#0412.png,N/A
steve,6,0.42,87.65,83.31,58.90,49.50,69.88,Steve#5376,Steve,c:/Users/dmarc/OneDrive/Documents/Github/Personal Projects/TritonRL-ScoreBot/playercard_imgs/Steve#5376.png,N/A
pacific_gull,7,0.52,86.49,84.32,64.00,59.60,46.71,N/A,N/A,N/A,N/A
greensleeves,8,0.55,84.46,83.21,34.47,83.65,41.12,N/A,N/A,N/A,N/A


In [10]:
dd = df.to_dict('index')

In [12]:
dd['wasurekaketeta']

{'ranking': 15,
 'win rate': 0.86,
 'Overall': 79.74,
 'Offense': 76.39,
 'Defense': 32.17,
 'Aggression': 66.22,
 'Speed': 39.33,
 'discord_username': 'When The Night#9055',
 'playercard_name': 'Honesty',
 'img_filepath': 'N/A',
 'rank': 'N/A'}

In [14]:
# TODO 
# Fix the Rank Images, need to be uploaded to discord and use discord link to get image. 
# Fix the bot commands 
# 
# Long Term 
# Have people populate their respective playercard names, ranks, and images
# Regenerate playercards once I have this data. 


In [14]:
patches = ['wasurekaketeta']

In [15]:
# FIFA playercard generation 
for player in dd.keys():
     if player in patches:
        club = default_logo

        nation_key = str(dd[player]['rank'])
        # Check to see if nation key is 'N/A', if it isnt 'N/A', get image fp from the dictionary.
        if nation_key != 'N/A':
            nation = rank_images[nation_key]
        else:
            nation = default_rank

        avatar_key = dd[player]['img_filepath']
        # Check to see if avatar key is 'N/A', if it isnt 'N/A', get image fp from the dictionary.
        if avatar_key != 'N/A':
            avatar = avatar_key
            avatar_flag = True
        else:
            avatar = default_img
            avatar_flag = False

        card_name = dd[player]['playercard_name']
        # Check to see if player updated their playercard name, otherwise use their ballchasing name. 
        # This will also be the name of the playercard file after it is saved locally. 
        if card_name != 'N/A':
            card_name = card_name
        else:
            card_name = player

        # Now we have if they used !update_name, then the file will be saved as their specified name
        # Otherwise, the file is saved under their ballchasing name. In order to access it, they need to use 
        # the '!update_name' command

        position_ovr = 'OVR'
        rating = str(int(dd[player]['Overall']))
        off_stat = str(int(dd[player]['Offense']))
        def_stat = str(int(dd[player]['Defense']))
        agg_stat = str(int(dd[player]['Aggression']))
        spd_stat = str(int(dd[player]['Speed']))
        win_stat = str(int(dd[player]['win rate']*100))
        rank_stat = str(int(dd[player]['ranking']))

        # Let the webpage load
        sleepy(2)
        # Select the correct card for the player
        browser.find_element_by_xpath('//*[@id="card_selector_btn"]').click()
        
        sleepy(1)
        if int(rank_stat) <= 5:
            tier = tiered_playercards['legend']
        if int(rank_stat) > 5 and int(rank_stat) <= 15:
            tier = tiered_playercards['champ']
        if int(rank_stat) > 15 and int(rank_stat) <=25:
            tier = tiered_playercards['gold']
        if int(rank_stat) > 25 and int(rank_stat) <= 35:
            tier = tiered_playercards['silver']
        if int(rank_stat) > 35 and int(rank_stat) <= 50:
            tier = tiered_playercards['bronze']
        
        browser.find_element_by_xpath(tier).click()
       
        sleepy(2)

        # Upload the players image
        browser.find_element_by_xpath('//*[@id="form-image-type-upload"]').click()
        browser.find_element_by_xpath(choose_file).clear()
        browser.find_element_by_xpath(choose_file).send_keys(avatar)
        sleepy(2)

        # Scale the players image
        if avatar_flag == True: 
            browser.find_element_by_xpath('//*[@id="form-card-upload-size"]').click()
            browser.find_element_by_xpath('//*[@id="form-card-upload-size"]').clear()
            browser.find_element_by_xpath('//*[@id="form-card-upload-size"]').send_keys('80')

        # Sends text to the field corresponding to player name
        browser.find_element_by_xpath(input_name).clear()
        browser.find_element_by_xpath(input_name).send_keys(card_name)

        # Enter the players overall rating
        browser.find_element_by_xpath(overall_rating).clear()
        browser.find_element_by_xpath(overall_rating).send_keys(rating)

        # Enter the players position (subsituted with overall text)
        browser.find_element_by_xpath(position).clear()
        browser.find_element_by_xpath(position).send_keys(position_ovr)

        # Enter the players home club
        browser.find_element_by_xpath(club_field).clear()
        browser.find_element_by_xpath(club_field).send_keys(club)
        sleepy(1)

        # Enter the players max rank during this tournament duration
        browser.find_element_by_xpath(nation_rank).clear()
        browser.find_element_by_xpath(nation_rank).send_keys(nation)
        sleepy(1)
        
        # Clear and populate attribute fields
        browser.find_element_by_xpath(offense).clear()
        browser.find_element_by_xpath(defense).clear()
        browser.find_element_by_xpath(aggression).clear()
        browser.find_element_by_xpath(speed).clear()
        browser.find_element_by_xpath(winrate).clear()
        browser.find_element_by_xpath(ranking).clear()
        browser.find_element_by_xpath(ostat).clear()
        browser.find_element_by_xpath(dstat).clear()
        browser.find_element_by_xpath(astat).clear()
        browser.find_element_by_xpath(sstat).clear()
        browser.find_element_by_xpath(wstat).clear()
        browser.find_element_by_xpath(rstat).clear()
        browser.find_element_by_xpath(offense).send_keys('OFF')
        browser.find_element_by_xpath(aggression).send_keys('AGG')
        browser.find_element_by_xpath(winrate).send_keys('W%')
        browser.find_element_by_xpath(defense).send_keys('DEF')
        browser.find_element_by_xpath(speed).send_keys('SPD')
        browser.find_element_by_xpath(ranking).send_keys('Rank')
        browser.find_element_by_xpath(ostat).send_keys(off_stat)
        browser.find_element_by_xpath(astat).send_keys(agg_stat)
        browser.find_element_by_xpath(wstat).send_keys(win_stat)
        browser.find_element_by_xpath(dstat).send_keys(def_stat)
        browser.find_element_by_xpath(sstat).send_keys(spd_stat)
        browser.find_element_by_xpath(rstat).send_keys(rank_stat)
       
        # Formatting and Download
        browser.find_element_by_xpath('//*[@id="createCardForm"]/div[18]/div/div/div/label[2]').click()
        sleepy(10)
        # browser.find_element_by_xpath('//*[@id="main_row"]/div[2]/div/div/button').click()
        



        # Give time for webpage to reset
         
        # Reload the webpage
        # browser.get('https://www.fifarosters.com/create-card')


In [16]:
# Pipeline working as of Monday February 1, 2021: Saved at 3:30pm

# Previous Work 

In [27]:
df

,index,win rate,Overall,Offense,Defense,Aggression,Speed
participant,,,,,,,
ix mini,0,0.66,99.00,100.00,44.39,72.86,90.61
invincible,1,0.43,98.64,83.53,37.51,100.00,84.85
shaunch,2,0.58,98.14,81.56,50.25,79.70,91.65
alex,3,0.52,96.13,96.61,55.78,63.56,76.20
pacific_gull,4,0.50,91.71,94.21,68.94,49.02,55.72
greensleeves,5,0.48,90.80,84.26,49.84,72.47,56.33
inf3ct3ds0ldi3r,6,0.55,89.97,79.41,83.27,45.82,49.82
steve,7,0.54,88.73,86.96,59.11,42.67,62.77
rj5588,8,0.35,88.13,58.62,63.25,26.32,100.00


In [7]:
# Find a way to match discord names to ballchasing.gg names

### DONE ### Merged to playerdataprocessing notebook
#  



# Load and Manage player card data

# Season Statistics
dataset = 'data/WINTER_playerstats.csv'
df = pd.read_csv(dataset, index_col=0)
df = df.reset_index()
df = df.set_index('participant', drop = True, inplace = False)

# Merge the player specified names
player_names = 'output/playercard_names.json'
def load_params(fp):
    with open(fp) as fh:
        param = json.load(fh)
    return param
names = load_params(player_names)
names_df = pd.DataFrame.from_dict(names,orient='index').reset_index(drop = False)
cols = {'index': 'discord_username', 0: 'ballchasing_username', 1: 'playercard_name'}
names_df = names_df.rename(columns = cols)
names_df.ballchasing_username = names_df.ballchasing_username.str.lower()
names_df= names_df.set_index('ballchasing_username', drop = True)
merged_names = df.merge(names_df, left_index = True, right_index = True, how = 'outer')  
merged_names = merged_names.reset_index(drop = False) # Reset the index here to preserve original playercard rank data
old_index = {'level_0': 'name_index'}
merged_names = merged_names.rename(columns = old_index)

# Merge the playercard image paths 
x = os.listdir("./playercard_imgs")
abs_path = start_directory + '\\playercard_imgs\\'
file_paths = [abs_path + i for i in x]
names = [name.strip('.png') for name in x]
fp_df = {'avatar_username': names, 'img_filepath': file_paths}
fp_df = pd.DataFrame(data = fp_df)
merged_names_paths = merged_names.merge(fp_df,  left_on = 'discord_username', right_on='avatar_username', how = 'outer')

# Merge the players associated rank 
player_ranks = 'output/player_ranks.json'
def load_params(fp):
    with open(fp) as fh:
        param = json.load(fh)
    return param
ranks = load_params(player_ranks)
cols = {'index': 'rank_username', 0: 'rank'}
rank_df = pd.DataFrame.from_dict(ranks,orient='index').reset_index(drop = False).rename({'index': 'player', '0': 'rank'})
rank_df = rank_df.rename(columns = cols)
merged_names_paths_ranks = merged_names_paths.merge(rank_df, left_on = 'discord_username', right_on='rank_username', how = 'outer')

# Clean Final DataFrame
cleaned_df = merged_names_paths_ranks.set_index('name_index')
cleaned_df = merged_names_paths_ranks.rename(columns={'index': 'ranking'}).sort_values('ranking')
cleaned_df['img_filepath'] = cleaned_df['img_filepath'].str.replace('\\', '/')
cleaned_df = cleaned_df.fillna('N/A')
cleaned_df = cleaned_df.drop(columns = ['avatar_username', 'rank_username']).reset_index(drop = True).set_index('name_index')


In [8]:
# Merge the playercard image paths 
x = os.listdir("./playercard_imgs")
abs_path = start_directory + '\\playercard_imgs\\'
file_paths = [abs_path + i for i in x]

names = [name.strip('.png') for name in x]

fp_df = {'player': names, 'img_filepath': file_paths}
fp_df = pd.DataFrame(data = fp_df)
# Lowercasing discord names for consistency
fp_df.player = fp_df.player.str.lower()

fp_df

,player,img_filepath
0,goofy,c:\Users\dmarc\OneDrive\Documents\Github\Personal Projects\TritonRL-ScoreBot\playercard_imgs\Goofy.png


In [9]:
merged_paths = df.merge(fp_df, left_index = True, right_on='player', how = 'outer').set_index('player')
merged_paths

,index,win rate,Overall,Offense,Defense,Aggression,Speed,img_filepath
player,,,,,,,,
ix mini,0,0.66,99.00,100.00,44.39,72.86,90.61,NaN
invincible,1,0.43,98.64,83.53,37.51,100.00,84.85,NaN
shaunch,2,0.58,98.14,81.56,50.25,79.70,91.65,NaN
alex,3,0.52,96.13,96.61,55.78,63.56,76.20,NaN
pacific_gull,4,0.50,91.71,94.21,68.94,49.02,55.72,NaN
greensleeves,5,0.48,90.80,84.26,49.84,72.47,56.33,NaN
inf3ct3ds0ldi3r,6,0.55,89.97,79.41,83.27,45.82,49.82,NaN
steve,7,0.54,88.73,86.96,59.11,42.67,62.77,NaN
rj5588,8,0.35,88.13,58.62,63.25,26.32,100.00,NaN


In [10]:
# Merge the players associated rank 
player_ranks = 'output/player_ranks.json'

def load_params(fp):
    with open(fp) as fh:
        param = json.load(fh)
    return param

ranks = load_params(player_ranks)

cols = {'index': 'player', 0: 'rank'}
rank_df = pd.DataFrame.from_dict(ranks,orient='index').reset_index(drop = False).rename({'index': 'player', '0': 'rank'})
rank_df = rank_df.rename(columns = cols)
# Lowercasing discord names for consistency
rank_df.player = rank_df.player.str.lower()

In [11]:
rank_df

,player,rank
0,goofy,Grand Champion I


In [12]:
merged_paths_ranks = merged_paths.merge(rank_df, left_index = True, right_on='player', how = 'outer').set_index('player')

In [13]:
merged_paths_ranks

,index,win rate,Overall,Offense,Defense,Aggression,Speed,img_filepath,rank
player,,,,,,,,,
ix mini,0,0.66,99.00,100.00,44.39,72.86,90.61,NaN,NaN
invincible,1,0.43,98.64,83.53,37.51,100.00,84.85,NaN,NaN
shaunch,2,0.58,98.14,81.56,50.25,79.70,91.65,NaN,NaN
alex,3,0.52,96.13,96.61,55.78,63.56,76.20,NaN,NaN
pacific_gull,4,0.50,91.71,94.21,68.94,49.02,55.72,NaN,NaN
greensleeves,5,0.48,90.80,84.26,49.84,72.47,56.33,NaN,NaN
inf3ct3ds0ldi3r,6,0.55,89.97,79.41,83.27,45.82,49.82,NaN,NaN
steve,7,0.54,88.73,86.96,59.11,42.67,62.77,NaN,NaN
rj5588,8,0.35,88.13,58.62,63.25,26.32,100.00,NaN,NaN


In [14]:
cleaned_df = merged_paths_ranks.rename(columns={'index': 'ranking'}).sort_values('ranking')
cleaned_df['img_filepath'] = cleaned_df['img_filepath'].str.replace('\\', '/')

In [15]:
cleaned_df

,ranking,win rate,Overall,Offense,Defense,Aggression,Speed,img_filepath,rank
player,,,,,,,,,
ix mini,0,0.66,99.00,100.00,44.39,72.86,90.61,NaN,NaN
invincible,1,0.43,98.64,83.53,37.51,100.00,84.85,NaN,NaN
shaunch,2,0.58,98.14,81.56,50.25,79.70,91.65,NaN,NaN
alex,3,0.52,96.13,96.61,55.78,63.56,76.20,NaN,NaN
pacific_gull,4,0.50,91.71,94.21,68.94,49.02,55.72,NaN,NaN
greensleeves,5,0.48,90.80,84.26,49.84,72.47,56.33,NaN,NaN
inf3ct3ds0ldi3r,6,0.55,89.97,79.41,83.27,45.82,49.82,NaN,NaN
steve,7,0.54,88.73,86.96,59.11,42.67,62.77,NaN,NaN
rj5588,8,0.35,88.13,58.62,63.25,26.32,100.00,NaN,NaN


Index(['ix mini', 'invincible', 'shaunch', 'alex', 'pacific_gull',
       'greensleeves', 'inf3ct3ds0ldi3r', 'steve', 'rj5588', 'luisito',
       'muffled', 'terminator', 'chyaboi', 'goofy', 'brictone', 'adamamango',
       'minimug', 'hotshot', 'krystopher', 'desolation', 'tgolden',
       'kameron (save replays)', 'fireyfrenchtoast', 'goose's ejection seat',
       'luke', 'scootleboot', 'casillas361', 'the monkey dude', 'monkenstein'],
      dtype='object', name='player')

In [18]:
def sleepy(sec):
    time.sleep(sec)
    return

In [19]:
# Merge the playercard image paths 
x = os.listdir("./default_imgs")
abs_path = start_directory + '\\default_imgs\\'
file_paths = [abs_path + i for i in x]

In [20]:
file_paths

['c:\\Users\\dmarc\\OneDrive\\Documents\\Github\\Personal Projects\\TritonRL-ScoreBot\\default_imgs\\default_player_avatar.png',
 'c:\\Users\\dmarc\\OneDrive\\Documents\\Github\\Personal Projects\\TritonRL-ScoreBot\\default_imgs\\default_player_rank.png']

In [21]:
dd = cleaned_df.to_dict('index')
dd['alex']

{'ranking': 3,
 'win rate': 0.52,
 'Overall': 96.13,
 'Offense': 96.61,
 'Defense': 55.78,
 'Aggression': 63.56,
 'Speed': 76.2,
 'img_filepath': nan,
 'rank': nan}

In [26]:
# # FIFA playercard generation 
# for player in dd.keys():
#     # if player == 'alex':
#         club = triton_logo

#         # TODO FIX
#         nation_key = str(dd[player]['rank'])
#         if nation_key == 'nan':
#             nation = default_rank
#         else:
#             nation = rank_images[nation_key]

#         # TODO FIX
#         avatar_key = dd[player]['img_filepath']
#         if np.isnan(avatar_key):
#             avatar = default_img
#         else:
#             avatar = rank_images[avatar_key]

#         position_ovr = 'OVR'
#         rating = str(int(dd[player]['Overall']))
#         off_stat = str(int(dd[player]['Offense']))
#         def_stat = str(int(dd[player]['Defense']))
#         agg_stat = str(int(dd[player]['Aggression']))
#         spd_stat = str(int(dd[player]['Speed']))
#         win_stat = str(int(dd[player]['win rate']*100))
#         rank_stat = str(int(dd[player]['ranking']))

#         # Let the webpage load
#         sleepy(2)
#         # Select the correct card for the player
#         browser.find_element_by_xpath('//*[@id="card_selector_btn"]').click()
        
#         sleepy(1)
#         if int(rank_stat) <= 5:
#             tier = tiered_playercards['legend']
#         if int(rank_stat) > 5 and int(rank_stat) <= 15:
#             tier = tiered_playercards['champ']
#         if int(rank_stat) > 15 and int(rank_stat) <=25:
#             tier = tiered_playercards['gold']
#         if int(rank_stat) > 25 and int(rank_stat) <= 35:
#             tier = tiered_playercards['silver']
#         if int(rank_stat) > 35 and int(rank_stat) <= 50:
#             tier = tiered_playercards['bronze']
        
#         browser.find_element_by_xpath(tier).click()
       
#         sleepy(2)

#         # Upload the players image
#         browser.find_element_by_xpath('//*[@id="form-image-type-upload"]').click()
#         browser.find_element_by_xpath(choose_file).clear()
#         browser.find_element_by_xpath(choose_file).send_keys(avatar)
#         sleepy(2)

#         # Sends text to the field corresponding to player name
#         browser.find_element_by_xpath(input_name).clear()
#         browser.find_element_by_xpath(input_name).send_keys(player)

#         # Enter the players overall rating
#         browser.find_element_by_xpath(overall_rating).clear()
#         browser.find_element_by_xpath(overall_rating).send_keys(rating)

#         # Enter the players position (subsituted with overall text)
#         browser.find_element_by_xpath(position).clear()
#         browser.find_element_by_xpath(position).send_keys(position_ovr)

#         # Enter the players home club
#         browser.find_element_by_xpath(club_field).clear()
#         browser.find_element_by_xpath(club_field).send_keys(club)

#         # Enter the players max rank during this tournament duration
#         browser.find_element_by_xpath(nation_rank).clear()
#         browser.find_element_by_xpath(nation_rank).send_keys(nation)
        
#         # Clear and populate attribute fields
#         browser.find_element_by_xpath(offense).clear()
#         browser.find_element_by_xpath(defense).clear()
#         browser.find_element_by_xpath(aggression).clear()
#         browser.find_element_by_xpath(speed).clear()
#         browser.find_element_by_xpath(winrate).clear()
#         browser.find_element_by_xpath(ranking).clear()
#         browser.find_element_by_xpath(ostat).clear()
#         browser.find_element_by_xpath(dstat).clear()
#         browser.find_element_by_xpath(astat).clear()
#         browser.find_element_by_xpath(sstat).clear()
#         browser.find_element_by_xpath(wstat).clear()
#         browser.find_element_by_xpath(rstat).clear()
#         browser.find_element_by_xpath(offense).send_keys('OFF')
#         browser.find_element_by_xpath(aggression).send_keys('AGG')
#         browser.find_element_by_xpath(winrate).send_keys('W%')
#         browser.find_element_by_xpath(defense).send_keys('DEF')
#         browser.find_element_by_xpath(speed).send_keys('SPD')
#         browser.find_element_by_xpath(ranking).send_keys('Rank')
#         browser.find_element_by_xpath(ostat).send_keys(off_stat)
#         browser.find_element_by_xpath(astat).send_keys(agg_stat)
#         browser.find_element_by_xpath(wstat).send_keys(win_stat)
#         browser.find_element_by_xpath(dstat).send_keys(def_stat)
#         browser.find_element_by_xpath(sstat).send_keys(spd_stat)
#         browser.find_element_by_xpath(rstat).send_keys(rank_stat)
       
#         # Formatting and Download
#         browser.find_element_by_xpath(thin_letters).click()
        
#         # browser.find_element_by_xpath(download).click()




#         # Give time for webpage to reset
#         sleepy(2)
#         # Reload the webpage
#         # browser.get('https://www.fifarosters.com/create-card')


ElementClickInterceptedException: Message: element click intercepted: Element <label class="btn btn-default active">...</label> is not clickable at point (212, 846). Other element would receive the click: <iframe frameborder="0" src="https://de3b9c8192b9f7a2935fdfd83b7b536c.safeframe.googlesyndication.com/safeframe/1-0-37/html/container.html" id="google_ads_iframe_/1254144/fifarosters_com-medrectangle-2_0" title="3rd party ad content" name="" scrolling="no" marginwidth="0" marginheight="0" width="728" height="90" data-is-safeframe="true" sandbox="allow-forms allow-popups allow-popups-to-escape-sandbox allow-same-origin allow-scripts allow-top-navigation-by-user-activation" data-google-container-id="c" style="border: 0px; vertical-align: bottom;" data-load-complete="true"></iframe>
  (Session info: chrome=88.0.4324.104)


In [ ]:
# Generation Pipeline 


# By default, playercards will be generated with data from the previous dataframe. 
# By default, an unlocked character image will be the 'locked character' avatar.

# For each player
# 